In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets

In [2]:
train_ratio = 0.8
rand_std = 0.030
learning_rate = 0.001
epoch_count = 100
report_period = 10
random_fix = True

In [3]:
iris_dataset = datasets.load_iris()

data = iris_dataset.data
target = iris_dataset.target
target_names = iris_dataset.target_names

#print("dimension: data{}, target{}, target_names{}".
#    format(data.shape, target.shape, target_names.shape))
#print(target_names)
#print(data[:5])
#print(target[-5:])

In [4]:
if random_fix: np.random.seed(1234)

data_count = len(data)
train_count = int(data_count * train_ratio)
test_count = data_count - train_count
    
indices = np.arange(data_count)
np.random.shuffle(indices)
    
train_data = data[indices[0:train_count]]
train_target = target[indices[0:train_count]]
    
test_data = data[indices[train_count:data_count]]
test_target = target[indices[train_count:data_count]]

In [5]:
input_dim, output_dim = 4, 1

def get_test_data():
    test_X = test_data
    test_Y = np.zeros([test_count, 1])
    
    for i in range(test_count):
        if test_target[i] == 0: test_Y[i, 0] = 1.0
            
    return test_X, test_Y

def get_train_data(batch_size, nth):
    global indices
    
    if nth == 0:
        indices = np.arange(train_count)
        np.random.shuffle(indices)
        
    from_idx = nth * batch_size
    to_idx = (nth + 1) * batch_size
    
    train_X = train_data[indices[from_idx:to_idx]]
    train_Y = np.zeros([batch_size, 1])
    
    for i in range(batch_size):
        k = indices[from_idx+i]
        if train_target[k] == 0: train_Y[i, 0] = 1.0
            
    return train_X, train_Y

In [6]:
def init_rand_normal(in_dim, out_dim):
    if not random_fix:
        init = tf.random_normal([in_dim, out_dim], stddev=rand_std)
    else:
        init_64 = np.random.normal(0, rand_std, [in_dim, out_dim])
        init = init_64.astype('float32')

    return init

In [7]:
hidden_dim = 10

x = tf.placeholder("float", [None, input_dim])
y = tf.placeholder("float", [None, output_dim])

if random_fix: np.random.seed(9876)

w_hid = tf.Variable(init_rand_normal(input_dim, hidden_dim))
b_hid = tf.Variable(tf.zeros([hidden_dim]))

w_out = tf.Variable(init_rand_normal(hidden_dim, output_dim))
b_out = tf.Variable(tf.zeros([output_dim]))

In [8]:
hidden = tf.nn.relu(tf.matmul(x, w_hid) + b_hid)
output = tf.matmul(hidden, w_out) + b_out

In [9]:
cross_entropy = \
  tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=output)
loss = tf.reduce_mean(cross_entropy)

#probs = tf.nn.sigmoid(output)
#estimate = tf.greater(probs, 0.5)
estimate = tf.greater(output, 0)
answer = tf.equal(y, 1.0)
correct = tf.cast(tf.equal(estimate, answer), "float")
accuracy = tf.reduce_mean(correct)

In [10]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

In [11]:
def train_and_test(batch_size=0):
    if batch_size == 0: batch_size = train_count
    batch_count = int(train_count / batch_size)
    
    test_X, test_Y = get_test_data()
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    if random_fix: np.random.seed(1945)
    
    for epoch in range(epoch_count):
        costs = []
        accs = []
        for n in range(batch_count):
            train_X, train_Y = get_train_data(batch_size, n)
            _, cost, acc = sess.run([train_op, loss, accuracy], \
                                    feed_dict={x:train_X, y:train_Y})
            costs.append(cost)
            accs.append(acc)
            
        if (epoch+1) % report_period == 0:
            acc = sess.run(accuracy, feed_dict={x:test_X, y:test_Y})
            print("Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f}". \
                  format(epoch+1, np.mean(costs), np.mean(accs), acc))
            
    final_acc = sess.run(accuracy, feed_dict={x:test_X, y:test_Y})
    print("\nFinal Test: final accuracy = {:5.3f}".format(final_acc))
    
    sess.close()

In [12]:
train_and_test(10)

Epoch 10: cost=0.689, accuracy=0.675/0.633
Epoch 20: cost=0.684, accuracy=0.675/0.633
Epoch 30: cost=0.678, accuracy=0.675/0.633
Epoch 40: cost=0.671, accuracy=0.675/0.633
Epoch 50: cost=0.663, accuracy=0.675/0.633
Epoch 60: cost=0.654, accuracy=0.675/0.633
Epoch 70: cost=0.642, accuracy=0.675/0.633
Epoch 80: cost=0.629, accuracy=0.675/0.633
Epoch 90: cost=0.615, accuracy=0.675/0.633
Epoch 100: cost=0.600, accuracy=0.675/0.633

Final Test: final accuracy = 0.633


In [13]:
train_and_test()

Epoch 10: cost=0.695, accuracy=0.033/0.000
Epoch 20: cost=0.695, accuracy=0.342/0.267
Epoch 30: cost=0.694, accuracy=0.658/0.600
Epoch 40: cost=0.693, accuracy=0.675/0.633
Epoch 50: cost=0.693, accuracy=0.675/0.633
Epoch 60: cost=0.692, accuracy=0.675/0.633
Epoch 70: cost=0.692, accuracy=0.675/0.633
Epoch 80: cost=0.691, accuracy=0.675/0.633
Epoch 90: cost=0.691, accuracy=0.675/0.633
Epoch 100: cost=0.690, accuracy=0.675/0.633

Final Test: final accuracy = 0.633


In [14]:
train_and_test(1)

Epoch 10: cost=0.608, accuracy=0.675/0.633
Epoch 20: cost=0.498, accuracy=0.675/0.633
Epoch 30: cost=0.359, accuracy=0.675/0.633
Epoch 40: cost=0.273, accuracy=0.942/0.900
Epoch 50: cost=0.229, accuracy=0.983/0.967
Epoch 60: cost=0.199, accuracy=1.000/1.000
Epoch 70: cost=0.176, accuracy=1.000/1.000
Epoch 80: cost=0.157, accuracy=1.000/1.000
Epoch 90: cost=0.142, accuracy=1.000/1.000
Epoch 100: cost=0.128, accuracy=1.000/1.000

Final Test: final accuracy = 1.000
